In [176]:
# Import Resale-Flat-Prices.csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as math
df = pd.read_csv("data/resale-flat-prices-sers-removed.csv", low_memory=False)

In [177]:
df.head(2)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0
1,1990-01,ANG MO KIO,3 ROOM,211,ANG MO KIO AVE 3,01 TO 03,67.0,NEW GENERATION,1977,46000.0


In [178]:
# df["year"] = df["month"].str.split("-").str[0]
# df["year"] = df["year"].astype(int)
# get first index where year > 2021
# df[df["year"] > 2020].index[0]

# Data Cleaning

In [179]:
# Combine Multi-Generation and Multi Generation categories in flat type
df["flat_type"] = df["flat_type"].replace("MULTI GENERATION", "MULTI-GENERATION")
df["flat_type"].unique()

array(['3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION'], dtype=object)

In [180]:
# Clean flat model column by capitalising and renaming
df["flat_model"] = df["flat_model"].replace("2-room", "2-ROOM")
df["flat_model"] = df["flat_model"].replace("2-ROOM", "2 ROOM")
df["flat_model"] = df["flat_model"].replace("3Gen", "3 GEN")
df["flat_model"] = df["flat_model"].replace("Adjoined flat", "ADJOINED FLAT")
df["flat_model"] = df["flat_model"].replace("Apartment", "APARTMENT")
df["flat_model"] = df["flat_model"].replace("Improved", "IMPROVED")
df["flat_model"] = df["flat_model"].replace("Improved-Maisonette", "IMPROVED-MAISONETTE")
df["flat_model"] = df["flat_model"].replace("IMPROVED-MAISONETTE", "IMPROVED MAISONETTE")
df["flat_model"] = df["flat_model"].replace("Maisonette", "MAISONETTE")
df["flat_model"] = df["flat_model"].replace("Model A", "MODEL A")
df["flat_model"] = df["flat_model"].replace("Model A-Maisonette", "MODEL A-MAISONETTE")
df["flat_model"] = df["flat_model"].replace("MODEL A-MAISONETTE", "MODEL A MAISONETTE")
df["flat_model"] = df["flat_model"].replace("New Generation", "NEW GENERATION")
df["flat_model"] = df["flat_model"].replace("Model A2", "MODEL A2")
df["flat_model"] = df["flat_model"].replace("MULTI GENERATION", "MULTI-GENERATION")
df["flat_model"] = df["flat_model"].replace("Multi Generation", "MULTI-GENERATION")
df["flat_model"] = df["flat_model"].replace("Premium Apartment", "PREMIUM APARTMENT")
df["flat_model"] = df["flat_model"].replace("Premium Apartment Loft", "PREMIUM APARTMENT LOFT")
df["flat_model"] = df["flat_model"].replace("Premium Maisonette", "PREMIUM MAISONETTE")
df["flat_model"] = df["flat_model"].replace("Simplified", "SIMPLIFIED")
df["flat_model"] = df["flat_model"].replace("Standard", "STANDARD")
df["flat_model"] = df["flat_model"].replace("Terrace", "TERRACE")
df["flat_model"] = df["flat_model"].replace("Type S1", "TYPE S1")
df["flat_model"] = df["flat_model"].replace("Type S2", "TYPE S2")

df["flat_model"].unique()

array(['NEW GENERATION', 'IMPROVED', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2 ROOM', 'IMPROVED MAISONETTE', 'MULTI-GENERATION',
       'PREMIUM APARTMENT', 'ADJOINED FLAT', 'PREMIUM MAISONETTE',
       'MODEL A2', 'DBSS', 'TYPE S1', 'TYPE S2', 'PREMIUM APARTMENT LOFT',
       '3 GEN'], dtype=object)

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878934 entries, 0 to 878933
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                878934 non-null  object 
 1   town                 878934 non-null  object 
 2   flat_type            878934 non-null  object 
 3   block                878934 non-null  object 
 4   street_name          878934 non-null  object 
 5   storey_range         878934 non-null  object 
 6   floor_area_sqm       878934 non-null  float64
 7   flat_model           878934 non-null  object 
 8   lease_commence_date  878934 non-null  int64  
 9   resale_price         878934 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 67.1+ MB


# Data Transformation

In [182]:
# Convert lease_commence_date to remaining lease at point of transaction
# lease commencing 1976 and sale in 1990 =  1976+99-1990 = 85 years remaining
df["remaining_lease"] = (df["lease_commence_date"] + 99 - df["month"].str[:4].astype(int))/99
df = df.drop(columns=['lease_commence_date'])


In [183]:
# Calculate inflated_adjusted_price 
hdb_resale_price_index = pd.read_csv("data/housing-and-development-board-resale-price-index-1q2009-100-monthly.csv", low_memory=False, index_col=0)
current_index = hdb_resale_price_index.tail(1)["index"].values[0]

# def get_adjusted_price(row):
#     time = row["month"].split("-")
#     quarter = time[0] + "-Q" + str((int(time[1]) - 1) // 3 + 1)
#     index = hdb_resale_price_index.loc[[quarter]].values[0][0]

# 	# get inflation adjusted price
#     adjusted_price = row["resale_price"] * (current_index / index)
#     return adjusted_price

# df["adjusted_price"] = df.apply(get_adjusted_price, axis=1)

# join with hdb_resale_price_index
df = df.join(hdb_resale_price_index, on="month", how="left", rsuffix="_index")
df["adjusted_price"] = df["resale_price"] * (current_index / df["index"])
df.drop(columns=["index", "resale_price", "month"], inplace=True)


In [184]:
# Add Region Data
# Central = ['BISHAN', 'BUKIT MERAH', 'BUKIT TIMAH', 'CENTRAL AREA', 'GEYLANG', 'KALLANG/WHAMPOA', 'MARINE PARADE', 'QUEENSTOWN', 'TOA PAYOH']
# East = ['BEDOK','PASIR RIS', 'TAMPINES']
# West = ['BUKIT BATOK', 'BUKIT PANJANG', 'CHOA CHU KANG', 'CLEMENTI', 'JURONG EAST', 'JURONG WEST']
# North East = ['ANG MO KIO','HOUGANG', 'PUNGGOL', 'SENGKANG','SERANGOON']
# North = ['SEMBAWANG', 'WOODLANDS', 'YISHUN',]

def get_region(row):
	town = row["town"]
	if town in ['BISHAN', 'BUKIT MERAH', 'BUKIT TIMAH', 'CENTRAL AREA', 'GEYLANG', 'KALLANG/WHAMPOA', 'MARINE PARADE', 'QUEENSTOWN', 'TOA PAYOH']:
		return "CENTRAL"
	elif town in ['BEDOK','PASIR RIS', 'TAMPINES']:
		return "EAST"
	elif town in ['BUKIT BATOK', 'BUKIT PANJANG', 'CHOA CHU KANG', 'CLEMENTI', 'JURONG EAST', 'JURONG WEST']:
		return "WEST"
	elif town in ['ANG MO KIO','HOUGANG', 'PUNGGOL', 'SENGKANG','SERANGOON']:
		return "NORTH-EAST"
	elif town in ['SEMBAWANG', 'WOODLANDS', 'YISHUN']:
		return "NORTH"
       
df["region"] = df.apply(get_region, axis=1)


In [185]:
# Add coordinate data from block_street_name_coords.json
df["block_street_name"] = df["block"].astype(str) + " " + df["street_name"].astype(str)
df = df.drop(columns=['block', 'street_name'])
block_street_name_coords = pd.read_json("data/block_street_name_coords.json")
block_street_name_coords = block_street_name_coords.transpose()

df = df.join(block_street_name_coords, on="block_street_name", how="left", rsuffix="_coords")
df.drop(columns=["block_street_name"], inplace=True)

In [186]:
# Add distance from Downtown Core planning area (CBD)
# Downtown Core planning area (CBD) = 1.286667, 103.853611
dg_mrt_lat = np.radians(1.286667)
dg_mrt_long = np.radians(103.853611)

df['distance_from_cbd'] = 6367 * 2 * np.arcsin(np.sqrt(np.sin((np.radians(df['latitude']) - dg_mrt_lat)/2)**2 + math.cos(math.radians(37.2175900)) * np.cos(np.radians(df['latitude'])) * np.sin((np.radians(df['longitude']) - dg_mrt_long)/2)**2))

In [187]:
# Transform storey_range to median of range (e.g. 01 TO 03 = 2)
def convert_to_median(row):
	storey_range = row["storey_range"].split(" TO ")
	median = (int(storey_range[0]) + int(storey_range[1])) / 2
	return median

df["median_storey"] = df.apply(convert_to_median, axis=1)
df = df.drop(columns=['storey_range'])

In [188]:
# Transform flat_type to ordinal encoding, town and flat_model to one-hot encoding
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

cols = 	["town", "region", "longitude", "latitude","distance_from_cbd",
         "flat_type", "flat_model", "floor_area_sqm", "median_storey", "remaining_lease",
         "adjusted_price"
    	]
df = df[cols]

pipeline = ColumnTransformer([ 
("o", OrdinalEncoder(), ["flat_type"]), 
("n", OneHotEncoder(sparse_output=False), ["town", "flat_model", "region"]), 
], remainder='passthrough', verbose_feature_names_out=False) 
pipeline.set_output(transform="pandas")
df = pipeline.fit_transform(df)

# Split Train-Test-Validation

In [189]:
# Split into train and test training data = pre-2021, test data = 2021 onwards
# test equal rows 819558 onward
test = df.iloc[819558:]
x_test = test.drop(columns=["adjusted_price"])
y_test = test["adjusted_price"]

train = df.iloc[:819558]
x_train = train.drop(columns=["adjusted_price"])
y_train = train["adjusted_price"]

In [196]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score 
 
lr = LinearRegression() 
lr.fit(x_train, y_train) 
print("R-sq of model: ", lr.score(x_train, y_train)) 
pred = lr.predict(x_test) 
print("RMSE: %.2f" % mean_squared_error(y_test, pred, squared=False)) 
print("R-sq of predictions: ", r2_score(y_test, pred)) 

R-sq of model:  0.8328513670534365
RMSE: 86066.31
R-sq of predictions:  0.7881633505883994


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878934 entries, 0 to 878933
Data columns (total 60 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   flat_type                          878934 non-null  float64
 1   town_ANG MO KIO                    878934 non-null  float64
 2   town_BEDOK                         878934 non-null  float64
 3   town_BISHAN                        878934 non-null  float64
 4   town_BUKIT BATOK                   878934 non-null  float64
 5   town_BUKIT MERAH                   878934 non-null  float64
 6   town_BUKIT PANJANG                 878934 non-null  float64
 7   town_BUKIT TIMAH                   878934 non-null  float64
 8   town_CENTRAL AREA                  878934 non-null  float64
 9   town_CHOA CHU KANG                 878934 non-null  float64
 10  town_CLEMENTI                      878934 non-null  float64
 11  town_GEYLANG                       8789

In [198]:
# print lr.coef_
lr.coef_

array([ 9.86197179e+04,  4.33947488e+04, -9.84210814e+03,  4.83466166e+04,
        9.29414533e+03, -2.73026235e+02, -6.12282757e+04,  9.20870773e+04,
       -3.85289556e+04, -2.71168242e+04,  6.65694623e+04, -5.77715654e+04,
       -1.54893651e+04,  1.16264706e+04, -1.55719406e+04, -5.61016201e+04,
        8.64782282e+04,  8.78832764e+02, -1.38133662e+04,  1.10093513e+04,
       -3.28006576e+04, -5.60319049e+04,  1.98953926e+04,  1.76292494e+04,
       -3.49456471e+04, -1.55290826e+04,  2.78347643e+04, -4.27510957e+04,
       -5.82076609e-11, -1.70332390e+05, -9.05398387e+04,  1.43016555e+05,
       -1.13316796e+05,  6.88201490e+03, -5.60886716e+04, -8.19207788e+04,
       -1.58182825e+04, -7.31291977e+04, -1.14237998e+05, -1.07699670e+05,
       -8.68777165e+04,  1.82776173e+05,  5.38278750e+04, -1.34616824e+05,
       -1.22942232e+05,  2.88788135e+05,  2.57462518e+05,  2.77518222e+05,
        5.03004590e+04,  8.66597406e+03, -2.04949760e+04, -2.20444948e+04,
       -1.64269624e+04, -

In [197]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)
rf.fit(x_train, y_train)
print("R-sq of model: ", rf.score(x_train, y_train))
pred = rf.predict(x_test)
print("RMSE: %.2f" % mean_squared_error(y_test, pred, squared=False))
print("R-sq of predictions: ", r2_score(y_test, pred))

R-sq of model:  0.922722432244333
RMSE: 67473.23
R-sq of predictions:  0.8698039646317295


In [ ]:
# plt.figure(figsize=(4, 24))
# heatmap = sns.heatmap(df.corr()[['adjusted_price']].sort_values(by='adjusted_price', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
# heatmap.set_title('Features Correlating with Inflation Adjusted Resale Price', fontdict={'fontsize':12}, pad=16);

In [ ]:
# Visualise adjusted_price per sqm on Singapore map
# import matplotlib.colors as colors
def visualise(df, vmin, vmax):
    
    df_sorted = df.sort_values(by='price_per_sqm')
    x = df_sorted['longitude']
    y = df_sorted['latitude']
    c = df_sorted['price_per_sqm'] 

    plt.rcParams['figure.figsize'] = [20, 10]
    plt.rcParams['figure.dpi'] = 100 

    # add image of singapore map
    img = plt.imread('data/3247px-Singapore_location_map_(main_island).svg.png')
    plt.imshow(img, extent=[103.557, 104.131, 1.129, 1.493])

    # set axes limits
    plt.xlim(103.62, 104.03)
    plt.ylim(1.23, 1.465)
    

    # Set axes titles
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.scatter(x, y, s=0.01, c=c, cmap='OrRd', 
                norm=colors.Normalize(vmin=vmin,vmax=vmax), alpha=0.8)
    cbar = plt.colorbar()
    cbar.set_label('Price per sqm', rotation=270, labelpad=20)
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.show()

# df2 = df
# df2["price_per_sqm"] = df2["adjusted_price"] / df2["floor_area_sqm"]
# visualise(df2, df2["price_per_sqm"].quantile(0.10), df2["price_per_sqm"].quantile(0.90))

### 